# Liage entre dicotopo et le pouillé

In [ ]:
#On importe nos dataframes nécessaires à la réinjection

import pandas as pd
import re
from unidecode import unidecode

#On importe les différents fichiers nécessaires. 

liage_po = pd.read_csv('../../utils/pouille/out/linking_out/liage_po7.csv', dtype=str)

In [ ]:
# On ne garde que les départements utiles à notre liage

dpt_list = liage_po['dpt_code'].unique().tolist()
dpt_list.remove("none")
dpt_list.sort()
print(dpt_list)  

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import re

# On définit le folder path
main_folder = '../../data'

#les fichiers avec les ID dicotopo sont tous appelés output7.xml.
def is_valid_xml_filename(file_path):
    if os.path.basename(file_path) == 'output7.xml':
        try:
            tree = ET.parse(file_path)
            root = tree.getroot()
            dep_attribute = root.attrib.get('dep')
            if dep_attribute and dep_attribute.isdigit() and dep_attribute in dpt_list:
                return True
        except Exception as e:
            pass
    
    return False

# On crée une liste pour y mettre le data 
data = []

# on cherche nos output7.xml
for root_dir, _, files in os.walk(main_folder):
    for filename in files:
        file_path = os.path.join(root_dir, filename)
        if is_valid_xml_filename(file_path):
            try:
                xml_tree = ET.parse(file_path)
                root_element = xml_tree.getroot()
            except ET.ParseError as e:
                print(f"Error parsing XML file {file_path}: {e}")
                continue  # Au cas où il y ait une erreur; pour que nous puissions voir

            #on extrait le old-id (pour la provenance) et le id (pour l'identifiant)   
            for article in root_element.findall('.//article'):
                article_old_id = article.get('old-id')
                article_id = article.get('id')
                
                # on extrait notre vedette
                vedette = article.find('./vedette/sm').text if article.find('./vedette/sm') is not None else ''

                # on extrait tout de notre définition en trois colonnes; typologie, insee et localisation
                definition_typologie = ''
                definition_localisation = ''
                insee = ''
                definition_elements = article.findall('./definition/*')
                for element in definition_elements:
                    if element.tag == 'typologie':
                        definition_typologie = element.text
                    elif element.tag == 'localisation':
                        localisation_content = ET.tostring(element, encoding='unicode')
                        localisation_match = re.search(r'<localisation>(.*?)<\/localisation>', localisation_content)
                        if localisation_match:
                            definition_localisation = re.sub(r'<[^>]*>', '', localisation_match.group(1))
                        insee_elem = element.find('./commune[@insee]')
                        if insee_elem is not None:
                            insee = insee_elem.get('insee')

                data.append({
                    'id': article_id,
                    'old-id' : article_old_id,
                    'vedette': vedette,
                    'definition_typologie': definition_typologie,
                    'definition_localisation': definition_localisation,
                    'insee': insee
                })

# on crée un dataframe
columns = ['id', 'old-id', 'vedette', 'definition_typologie', 'definition_localisation', 'insee']
places = pd.DataFrame(data, columns=columns)

# on print
print(places)

In [ ]:
#On applique la fonction voulue par Olivier pour supprimer les accents etc

import re
from unidecode import unidecode

def replace_special_characters(text):
    text = unidecode(text)
    text = re.sub(r'[-\'(),.]', ' ', text)  # Remlace également les parenthèses et les virgules
    return text


liage_po['vedette'] = liage_po['vedette'].astype(str)
places['vedette'] = places['vedette'].astype(str)

liage_po['vedette'] = liage_po['vedette'].apply(replace_special_characters)
places['vedette'] = places['vedette'].apply(replace_special_characters)

In [ ]:
file_path = '../../utils/pouille/resources/tokens_dicotopo.txt'

with open(file_path, 'r') as file:
    tokens = file.read() 

In [ ]:
def filter_words(text, tokens):
    text = replace_special_characters(text)  # Appliquer la fonction replace_special_characters
    words = text.split()
    words = [word for word in words if not re.search(r'\b{}\b'.format(re.escape(word.lower())), tokens)]
    filtered_text = ' '.join(words).strip() 
    return filtered_text

liage_po['vedette'] = liage_po['vedette'].apply(lambda x: filter_words(x, tokens=tokens))
places['vedette'] = places['vedette'].apply(lambda x: filter_words(x, tokens=tokens))

In [ ]:
places

In [ ]:
# dataframe à charger
liages_df = pd.DataFrame()

In [ ]:
columns_to_drop_po = ['localisationde', 'dpt_code', 'canton_code', 'method', 'reference']

# Renommer la colonne 'vedette' en 'label'
places.rename(columns={'vedette': 'label'}, inplace=True)

In [ ]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = liage_po[liage_po['localisationco'] == 'none']
communes_df = communes_df.drop(columns=columns_to_drop_po)

# Communes exact match

In [ ]:
#on lie par exact match vedette / label

#on enlève les lignes sans insee_code de liage_po7: elles ne servent à rien
communes_df_nonan = communes_df.dropna(subset=['insee_code'])

liage_exact_communes = pd.merge(communes_df_nonan,
                      places,
                      how='inner',
                      left_on=['insee_code','vedette'],
                      right_on=['insee', 'label'])


# Check for duplicate entries
duplicates = liage_exact_communes.duplicated(['article_id'], keep=False)
if duplicates.any():
    liage_exact_communes = liage_exact_communes[~duplicates]
    
# Add a 'method' column with value 'dpt_exact'
liage_exact_communes['method_dicotopo'] = 'communes_exact'


# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_exact_communes]).drop_duplicates()

In [ ]:
liages_df

# Lieux dans des communes exact match

In [ ]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

localisationco_df = liage_po[liage_po['localisationco'] != 'none']
localisationco_df = localisationco_df.drop(columns=columns_to_drop_po)

In [ ]:
#on lie par exact match vedette de la localisationco / label

#on enlève les lignes sans insee_code de liage_po7: elles ne servent à rien
localisationco_df_nonan = localisationco_df.dropna(subset=['insee_code'])

liage_exact_localisationco = pd.merge(localisationco_df_nonan,
                      places,
                      how='inner',
                      left_on=['insee_code','vedette'],
                      right_on=['insee', 'label'])


# Check for duplicate entries
duplicates = liage_exact_localisationco.duplicated(['article_id'], keep=False)
if duplicates.any():
    liage_exact_localisationco = liage_exact_localisationco[~duplicates]
    
# Add a 'method' column with value 'dpt_exact'
liage_exact_localisationco['method_dicotopo'] = 'localisationco_exact'


# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_exact_localisationco]).drop_duplicates()

In [ ]:
liages_df.agg(['nunique', 'count', 'size'])

In [ ]:
liages_df

# Fuzzy match communes

In [ ]:
import pandas as pd
from thefuzz import fuzz

communes_df_nonan = communes_df_nonan[~communes_df_nonan.article_id.isin(liages_df['article_id'])]

liage_fuzzy_communes = pd.merge(communes_df_nonan,
                                places,
                                how='inner',
                                left_on=['insee_code'],
                                right_on=['insee'])

# Define a threshold for similarity (50%: tout ce qui est plus bas donne des résultats vraiment trop erronnés.)
similarity_threshold = 50

# Filter communes based on fuzzy matching with insee_code
def fuzzy_match(row):
    vedette = row['vedette']
    label = row['label']

    if isinstance(vedette, str) and not pd.isnull(vedette) and isinstance(label, str) and not pd.isnull(label):
        # Calculate the fuzzy similarity score
        similarity_score = fuzz.ratio(vedette, label)
        if similarity_score >= similarity_threshold:
            return True

    return False

liage_fuzzy_communes = liage_fuzzy_communes[liage_fuzzy_communes.apply(fuzzy_match, axis=1)]

# Add a 'method' column with value 'dicotopo_fuzzy'
liage_fuzzy_communes['method_dicotopo'] = 'communes_fuzzy'

# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_fuzzy_communes]).drop_duplicates()

In [ ]:
liage_fuzzy_communes.agg(['nunique', 'count', 'size'])

In [ ]:
liage_fuzzy_communes

# Fuzzy match localisationco

In [ ]:

localisationco_df_nonan = localisationco_df_nonan[~localisationco_df_nonan.article_id.isin(liages_df['article_id'])]

liage_fuzzy_localisationco = pd.merge(localisationco_df_nonan,
                                places,
                                how='inner',
                                left_on=['insee_code'],
                                right_on=['insee'])


import pandas as pd
from thefuzz import fuzz

# Define a threshold for similarity (50%)
similarity_threshold = 50

# Filter localisationco_df based on fuzzy matching with insee_code
def fuzzy_match(row):
    vedette = row['vedette']
    label = row['label']

    if isinstance(vedette, str) and not pd.isnull(vedette) and isinstance(label, str) and not pd.isnull(label):
        # Calculate the fuzzy similarity score
        similarity_score = fuzz.ratio(vedette, label)
        if similarity_score >= similarity_threshold:
            return True

    return False

# Assuming you have defined liage_fuzzy_localisationco earlier
liage_fuzzy_localisationco = liage_fuzzy_localisationco[liage_fuzzy_localisationco.apply(fuzzy_match, axis=1)]

# Add a 'method' column with value 'dicotopo_fuzzy'
liage_fuzzy_localisationco['method_dicotopo'] = 'localisationco_fuzzy'

# Update the linked_places_df dataframe
liages_df = pd.concat([liages_df, liage_fuzzy_localisationco]).drop_duplicates()

In [ ]:
liage_fuzzy_localisationco.agg(['nunique', 'count', 'size'])

# Exportation

In [ ]:
#on append les deux dataframes ensemble

new_rows_df = liage_po[~liage_po['article_id'].isin(liages_df['article_id'])]

if 'method_dicotopo' not in new_rows_df.columns:
    new_rows_df['method_dicotopo'] = 'nulle'

liages_df = liages_df.append(new_rows_df, ignore_index=True)

In [ ]:
liages_df.agg(['nunique', 'count', 'size'])

In [ ]:
#on ajoute les lignes de notre pouillé qui n'ont pas eu de match avec dicotopo pour vérification manuelle

remaining_rows = liage_po[~liage_po['article_id'].isin(liages_df['article_id'])]
updated_liages_df = pd.concat([liages_df, remaining_rows], ignore_index=True)

#on sort par article_id pour plus de clarté
updated_liages_df = updated_liages_df.sort_values('article_id')
updated_liages_df = updated_liages_df.reset_index(drop=True)

In [ ]:
updated_liages_df

In [ ]:
updated_liages_df.agg(['nunique', 'count', 'size'])

In [ ]:
# Sauvegarder le dataframe fusionné
updated_liages_df.to_csv('../../utils/pouille/out/reinjection_out/po7_dicotopo.csv', index=False)

In [ ]:
#partie pour plus tard

# Réinsertion dans le fichier XML

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

# Charger le fichier XML
tree = ET.parse('../../utils/pouille/out/linking_out/PO_t7_modified.xml')
root = tree.getroot()
# Replace nan values with empty string in the DataFrame (juste pour le temps où olivier corrige)
liages_df = liages_df.replace({np.nan: ""})

In [ ]:
# Pour chaque article dans le XML
for article in root.findall('article'):
    # Récupérer l'article_id
    article_id = article.get('old-id')

    # Vérifier si l'article_id est présent dans le DataFrame
    if article_id in liages_df['article_id'].values:
        # Récupérer la ligne correspondante dans le DataFrame
        liage_row = liages_df.loc[liages_df['article_id'] == article_id]

        # Récupérer la valeur de place_id
        dicotopo_code = liage_row['id'].values[0]

        # Ajouter l'attribut 'dicotopo' à la balise article
        article.set('dicotopo', dicotopo_code)

In [ ]:
# Enregistrer le fichier XML modifié
tree.write('../../utils/pouille/out/reinjection_out/po7_dicotopo.xml', encoding='UTF-8', xml_declaration=True)